In [1]:
import pandas as pd
import datetime
import numpy as np
from copy import deepcopy
import cProfile
import sys
from collections import defaultdict
from tqdm import tqdm 
import pickle
from bisect import bisect_left
import random
from tqdm import tqdm
import random
import os
import logging


from data_structures.graph import TransportGraph, ContactionTransportGraph
from algorithms.forward_search import FCH
from algorithms.dijkstra import Dijkstra
from data_structures.timetable import Timetable
from algorithms.connection_scan_algorithm import ConnectionScanAlgorithm

# Select city

In [2]:
CITY = 'luxembourg'

# Build Transport Graphs

In [3]:
# Download transport connection
transport_connections = pd.read_csv(F'data/{CITY}/network_temporal_day.csv', sep=';')
transport_connections = transport_connections[
    transport_connections['from_stop_I']!=transport_connections['to_stop_I']]

# Download walking connections
walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

#Additional limit 
walk_connections = walk_connections[walk_connections['d_walk'] < 600]

#Build tg graph
tg = TransportGraph(transport_connections=transport_connections, walk_connections=walk_connections)

In [4]:
NODES = tg.nodes

# Build CH-graph

In [5]:
ch_tg = tg.contraction_hierarchy()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1367/1367 [00:26<00:00, 51.48it/s]


# Precompute geometrical containers

In [6]:
ch_tg.geometrical_container()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1367/1367 [00:00<00:00, 14436.57it/s]


# Precompute timetable nodes

### For standard graph

In [7]:
tg_ttn = deepcopy(tg)
tg_ttn_fc_ascending = deepcopy(tg)
tg_ttn_fc_descending = deepcopy(tg)

tg_ttn.optimize_binary_search()

tg_ttn_fc_ascending.fractional_cascading_precomputation('ascending')
tg_ttn_fc_descending.fractional_cascading_precomputation('descending')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1367/1367 [00:00<00:00, 3563.96it/s]


### For CH-graph

In [8]:
ch_tg_ttn = deepcopy(ch_tg)
ch_tg_ttn_fc_ascending = deepcopy(ch_tg)
ch_tg_ttn_fc_descending = deepcopy(ch_tg)
ch_tg_ttn_fc_hierarchy = deepcopy(ch_tg)

ch_tg_ttn.optimize_binary_search()
ch_tg_ttn_fc_ascending.fractional_cascading_precomputation('ascending')
ch_tg_ttn_fc_descending.fractional_cascading_precomputation('descending')
ch_tg_ttn_fc_hierarchy.fractional_cascading_precomputation('contraction_hierarchy')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1367/1367 [00:09<00:00, 145.33it/s]


# Prepare data for Connection Scan Algorithm

In [9]:
timetable = Timetable(transport_connections=transport_connections, walk_connections=walk_connections)

# Save Preprocessed Data Structures

In [10]:
# os.mkdir(f'graph/')


# Classic Transport graph
pickle.dump(tg, open(f'graph/{CITY}_tg.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(tg_ttn, open(f'graph/{CITY}_tg_ttn.pkl', 'wb'), 
        pickle.HIGHEST_PROTOCOL)
pickle.dump(tg_ttn_fc_ascending, open(f'graph/{CITY}_tg_ttn_fc_ascending.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(tg_ttn_fc_descending, open(f'graph/{CITY}_tg_ttn_fc_descending.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# CH-graph
pickle.dump(ch_tg, open(f'graph/{CITY}_ch_tg.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg_ttn, open(f'graph/{CITY}_ch_tg_ttn.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg_ttn_fc_ascending, open(f'graph/{CITY}_ch_tg_ttn_fc_ascending.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg_ttn_fc_descending, open(f'graph/{CITY}_ch_tg_ttn_fc_descending.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg_ttn_fc_hierarchy, open(f'graph/{CITY}_ch_tg_ttn_fc_hierarchy.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# CSA precomputation
pickle.dump(timetable, open(f'graph/{CITY}_csa.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

### Delete datastructures to reduce memory udage for test

In [11]:
del tg, tg_ttn, tg_ttn_fc_ascending, tg_ttn_fc_descending, ch_tg, ch_tg_ttn, ch_tg_ttn_fc_ascending, ch_tg_ttn_fc_descending, ch_tg_ttn_fc_hierarchy,timetable 

# Compare solutions

In [2]:
N = 1000
algorithms = [ 'tg', 'tg_ttn', 'tg_ttn_fc_ascending', 'tg_ttn_fc_descending',
         'ch_tg', 'ch_tg_ttn', 'ch_tg_ttn_fc_ascending', 'ch_tg_ttn_fc_descending', 'ch_tg_ttn_fc_hierarchy',
         'csa']
#results = []


test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                       transport_connections['dep_time_ut'].max()) for i in range(N)],
         'start_node' : [random.sample(NODES, 1)[0] for i in range(N)], 
          'end_node' : [random.sample(NODES, 1)[0] for i in range(N)]
         })
duration = {x: [] for x in algorithms}
explored_nodes = {x: [] for x in algorithms}
trip_duration = {x: [] for x in algorithms}

for algorithm in algorithms:
    graph = pickle.load(open(F'graph/{CITY}_{algorithm}.pkl', 'rb'))
    for index, row in test_data.iterrows():
        if algorithm == 'ch_tg':
            pathfinding = FCH(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'ch_tg_ttn':
            pathfinding = FCH(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, 
                                             time_table_nodes='cst')
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm in ['ch_tg_ttn_fc_ascending', 'ch_tg_ttn_fc_descending']: 
            pathfinding = FCH(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, 
                                             time_table_nodes='fc')
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
            
        elif algorithm == 'ch_tg_ttn_fc_hierarchy':
            pathfinding = FCH(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, 
                                             time_table_nodes='fc_chs')
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
            
        elif algorithm == 'tg':
            pathfinding = Dijkstra(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'tg_ttn':
            pathfinding = Dijkstra(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, 
                                             time_table_nodes='cst')
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm in ['tg_ttn_fc_ascending', 'tg_ttn_fc_descending']:
            pathfinding = Dijkstra(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, 
                                             time_table_nodes='fc')
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'csa':
            pathfinding = ConnectionScanAlgorithm(graph=graph,
                  start_time=row['start_time'],
                  start_node=row['start_node'], 
                  end_node=row['end_node'])
            path = pathfinding.shortest_path()
            duration[algorithm].append(path['duration'])
            
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])


NameError: name 'pd' is not defined

# Results

In [13]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.std(duration[algorithm]))

tg 7.775 4.9534205353472665
tg_ttn 7.094 4.6285163929708615
tg_ttn_fc_ascending 6.925 4.525635314516626
tg_ttn_fc_descending 7.518 5.072245656511522
ch_tg 4.343 2.3641808306472663
ch_tg_ttn 2.471 1.5255028679094642
ch_tg_ttn_fc_ascending 3.871 2.133625787245739
ch_tg_ttn_fc_descending 3.908 2.145585234848525
ch_tg_ttn_fc_hierarchy 3.316 1.9626879527831211
csa 5.404 4.164706952475768
